In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = 'cuda'

import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from pytorchtools import EarlyStopping
assert(torch.cuda.is_available())

In [2]:
def clear_str(text):
    return text.lower().replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('\'', '').replace('"', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').split(' ')

def load_news(path):
    news_dict = {} # index -> news
    news_list = [] # index -> news
    newsid_dict = {} # newsid -> index
    word_dict = {'<PAD>': 0, '<OOV>': 1}
    cate_dict = {'<PAD>': 0, '<OOV>': 1}
    ent_dict = {'<PAD>': 0, '<OOV>': 1}
    
    with open(path, 'r') as f:
        for line in f.readlines():
            news_id, category, subcategory, title, abstract, \
                url, title_entities, abstract_entities = line.strip().split('\t')
            title = clear_str(title)
            abstract = clear_str(abstract)
            entities = json.loads(title_entities) + json.loads(abstract_entities)
            ent_words = [clear_str(ent['Label']) for ent in entities] # list(list(str))
            ent_ids = [ent['WikidataId'] for ent in entities] # list(str): ['Q80976', 'Q43274', 'Q9682']
            for word in title + abstract:
                if word not in word_dict:
                    word_dict[word] = len(word_dict)
            for item in ent_words:
                for word in item:
                    if word not in word_dict:
                        word_dict[word] = len(word_dict)
            for ent in ent_ids:
                if ent not in ent_dict:
                    ent_dict[ent] = len(ent_dict)
            if category not in cate_dict:
                cate_dict[category] = len(cate_dict)
            if subcategory not in cate_dict:
                cate_dict[subcategory] = len(cate_dict)
            if news_id not in newsid_dict:
                newsid_dict[news_id] = len(newsid_dict)
                news_list.append([category, subcategory, title, abstract, ent_words, ent_ids])
    print(len(news_list))
    return news_list, newsid_dict, word_dict, cate_dict, ent_dict

news_list, newsid_dict, word_dict, cate_dict, ent_dict = load_news('/data/Recommend/MIND/small_news.tsv')

65238


In [3]:
max_title = 30
max_body = 100
max_ent_words = 10 # 每个entity最多保留10个单词
max_ent = 5 # 每条新闻最多保留5个entity
def map_news_input(news_list, word_dict, cate_dict):
    n_news = len(news_list)
    titles = np.zeros((n_news, max_title), dtype = 'int32')
    bodys = np.zeros((n_news, max_body), dtype = 'int32')
    cates = np.zeros((n_news,1), dtype = 'int32')
    subcates = np.zeros((n_news,1), dtype = 'int32')
    ent_words = np.zeros((n_news, max_ent, max_ent_words), dtype = 'int32')
    ent_ids = np.zeros((n_news, max_ent), dtype = 'int32')
    for i in range(n_news):
        category, subcategory, title, abstract, ent_word, ent_id = news_list[i]
        titles[i, :len(title)] = [word_dict[word] for word in title[:max_title]]
        bodys[i, :len(abstract)] = [word_dict[word] for word in abstract[:max_body]]
        cates[i] = cate_dict[category]
        subcates[i] = cate_dict[subcategory]
        ent_ids[i, :len(ent_id)] = [ent_dict[ent] for ent in ent_id[:max_ent]]
        for j in range(min(len(ent_word), max_ent)):
            ent_words[i, j, :len(ent_word[j])] = [word_dict[word] for word in ent_word[j][:max_ent_words]]
    news_info = np.concatenate((titles, bodys, cates, subcates), axis = 1)
    news_ents = np.concatenate((ent_ids.reshape(n_news, max_ent, 1), ent_words), axis = 2)
    print(news_info.shape, news_ents.shape)
    return news_info, news_ents # index -> news_info

'''
news_list: original news
news_info: mapped news(word ids)
'''
news_info, news_ents = map_news_input(news_list, word_dict, cate_dict)

(65238, 132) (65238, 5, 11)


In [4]:
def load_glove(word_to_ix, dim = 100):
    if dim == 100:
        path = '/data/pretrained/Glove/glove.6B.100d.txt'
    elif dim == 300:
        path = '/data/pretrained/Glove/glove.840B.300d.txt'
    word_emb = []
    word_emb = np.zeros((len(word_to_ix), dim), dtype = float)
    with open(path, 'r') as f:
        for line in f:
            data = line.strip().split(' ') # [word emb1 emb2 ... emb n]
            word = data[0]
            if word in word_to_ix:
                word_emb[word_to_ix[word]] = [float(i) for i in data[1:]]
    print(word_emb.shape)
    return torch.tensor(word_emb, dtype = torch.float)

word_emb = load_glove(word_dict, 300)
cate_emb = load_glove(cate_dict, 100)

(82568, 300)
(282, 100)


In [5]:
def load_ent_emb(path):
    ent_emb = []
    ent_dict = {'<PAD>': 0, '<OOV>': 1}
    with open(path, 'r') as f:
        for line in f:
            data = line.strip().split('\t')
            ent_id = data[0]
            ent_dict[ent_id] = len(ent_dict)
            ent_emb.append([float(i) for i in data[1:]])
    ent_emb.insert(0, [0.] * len(ent_emb[0]))
    ent_emb.insert(0, [0.] * len(ent_emb[0]))
    ent_emb = torch.tensor(ent_emb, dtype = torch.float)
    print(ent_emb.shape)
    return ent_emb, ent_dict
ent_emb, ent_dict = load_ent_emb('/data/Recommend/MIND/small_entity_embedding.vec')

torch.Size([31453, 100])


In [6]:
def load_train_impression(path, newsid_dict): # train&dev
    logs = []
    with open(path, 'r') as f:
        for line in f:
            imp_id, user_id, time, history, impression = line.strip().split('\t')
            if history:
                history = [newsid_dict[news_id] for news_id in history.split(' ')]
            else:
                history = []
            positive = []
            negative = []
            for item in impression.split(' '):
                news_id, num = item.split('-')
                if num == '1':
                    positive.append(newsid_dict[news_id])
                else:
                    negative.append(newsid_dict[news_id])
            logs.append([history, positive, negative]) # indexs
    return logs

train_logs = load_train_impression('/data/Recommend/MIND/MINDsmall_train/behaviors.tsv', newsid_dict)
dev_logs = load_train_impression('/data/Recommend/MIND/MINDsmall_dev/behaviors.tsv', newsid_dict)

In [7]:
max_history = 50
def map_user(logs): # index -> history, 用 index 代表 user_id, train&dev
    n_user = len(logs)
    user_hist = np.zeros((n_user, max_history), dtype = 'int32') # index -> history
    for i in range(n_user):
        history, positive, negative = logs[i]
        n_hist = len(history)
        if n_hist == 0:
            continue
        user_hist[i, -n_hist:] = history[-max_history:]
    return user_hist   

train_user_hist = map_user(train_logs)
dev_user_hist = map_user(dev_logs)

In [8]:
neg_ratio = 4
def neg_sample(negative):
    if len(negative) < neg_ratio:
        return random.sample(negative * (neg_ratio // len(negative) + 1), neg_ratio)
    else:
        return random.sample(negative, neg_ratio)

def get_train_input(logs): # 和 map_user 使用同一个 log
    all_pos = [] # 每个 sample 的 pos
    all_neg = []
    user_id = [] # 每个 sample 的 user，用 index 表示，和 map_user 的结果对应
    for i in range(len(logs)):
        history, positive, negative = logs[i]
        for pos in positive:
            all_pos.append(pos)
            all_neg.append(neg_sample(negative))
            user_id.append(i)
    n_imps = len(all_pos)
    imps = np.zeros((n_imps, 1 + neg_ratio), dtype = 'int32')
    for i in range(len(all_pos)):
        imps[i, 0] = all_pos[i]
        imps[i, 1:] = all_neg[i]
    user_id = np.array(user_id, dtype = 'int32')
    labels = np.zeros((n_imps, 1 + neg_ratio), dtype = 'int32')
    labels[:, 0] = 1
    print(n_imps)
    return imps, user_id, labels

def get_dev_input(logs): # 和 map_user 使用同一个 log
    imps = []
    labels = []
    user_id = np.zeros((len(logs)), dtype = 'int32') # 每个 sample 的 user index，和 map_user 的结果对应
    for i in range(len(logs)):
        history, positive, negative = logs[i]
        imps.append(np.array(positive + negative, dtype = 'int32'))
        labels.append([1] * len(positive) + [0] * len(negative))
        user_id[i] = i
    print(len(logs))
    return imps, user_id, labels

train_datas, train_users, train_labels = get_train_input(train_logs)
dev_datas, dev_users, dev_labels = get_dev_input(dev_logs)
valid_datas, valid_users, valid_labels = get_train_input(dev_logs) # 用 train 的方法构造 dev_set

236344
73152
111383


In [9]:
class TrainDataset(Dataset):
    def __init__(self, imp_datas, imp_users, imp_labels, news_info, user_clicks, batch_size, news_ents = None, news_urls = None):
        self.imp_datas = imp_datas # (n_imps, 1 + k)
        self.imp_users = imp_users
        self.imp_labels = imp_labels
        self.news = news_info
        self.user_clicks = user_clicks
        self.batch_size = batch_size
        self.news_ents = news_ents
        self.news_urls = news_urls
        
        self.n_data = imp_datas.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.n_data / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, self.n_data)
        
        data_id = self.imp_datas[start: end] # (n_batch, 1 + k)
        data_news = self.news[data_id] # (n_batch, 1 + k, news_len)
        user_id = self.imp_users[start: end] # (n_batch)
        user_news_id = self.user_clicks[user_id] # (n_batch, n_hist)
        user_news = self.news[user_news_id] # (n_batch, n_hist, news_len)
        labels = self.imp_labels[start: end] # (n_batch, 1 + k)
        
        if self.news_ents is not None:
            samp_ents = self.news_ents[data_id]
            user_ents = self.news_ents[user_news_id]
            return data_news, user_news, labels, samp_ents, user_ents
        
        if self.news_urls is not None:
            samp_urls = self.news_urls[data_id]
            user_urls = self.news_urls[user_news_id]
            return data_news, user_news, labels, samp_urls, user_urls
        
        return data_news, user_news, labels
    
class DevDataset(Dataset): # data 和 label 是 list，每条数据不同长度
    def __init__(self, imp_datas, imp_users, imp_labels, news_info, user_clicks, batch_size):
        self.imp_datas = imp_datas # [imp1, imp2, ..., impn]
        self.imp_users = imp_users # (n_imps)
        self.imp_labels = imp_labels
        self.news = news_info
        self.user_clicks = user_clicks
        self.batch_size = batch_size
        
        self.n_data = len(imp_datas)
        
    def __len__(self):
        return int(np.ceil(self.n_data / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, self.n_data)
        
        data_ids = []
        data_news = [] # [(n_imp, news_len)]
        labels = [] # [(n_imp)]
        for i in range(start, end):
            data_id = self.imp_datas[i] # (n_imp)
            data_ids.append(data_id)
            # data_news.append(self.news[data_id]) # (n_imp, news_len)
            labels.append(self.imp_labels[i]) # (n_imp)
        user_id = self.imp_users[start: end] # (n_batch)
        user_news_id = self.user_clicks[user_id] # (n_batch, n_hist)
        # user_news = self.news[user_news_id] # (n_batch, n_hist, news_len)
        
        #return data_news, user_news, labels
        return data_ids, user_news_id, labels


n_batch = 16
train_dataset = TrainDataset(train_datas, train_users, train_labels, news_info, train_user_hist, n_batch, news_ents)
dev_dataset = DevDataset(dev_datas, dev_users, dev_labels, news_info, dev_user_hist, 64)
valid_dataset = TrainDataset(valid_datas, valid_users, valid_labels, news_info, dev_user_hist, n_batch, news_ents)

In [10]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [11]:
def encode_all_news(news_encoder, news_info, news_ents = None):
    print(news_info.shape, news_ents.shape)
    n_news = len(news_info)
    news_rep = []
    n_batch = 32
    for i in range((len(news_info) + n_batch - 1) // n_batch):
        batch_news = torch.tensor(news_info[i * n_batch: (i + 1) * n_batch], dtype = torch.long, device = 'cuda')
        batch_ents = torch.tensor(news_ents[i * n_batch: (i + 1) * n_batch], dtype = torch.long, device = 'cuda')
        print(batch_news.shape, batch_ents.shape)
        batch_rep = news_encoder(batch_news, batch_ents)
        print(batch_rep.shape)
        batch_rep = batch_rep.detach().cpu().numpy()
        news_rep.append(batch_rep)
    news_rep = np.concatenate(news_rep, axis = 0)
    return news_rep # (n_news, n_title, n_emb)

def encode_all_user(user_encoder, user_ids, user_hist, news_rep):
    user_rep = []
    with torch.no_grad():
        for _, batch in enumerate(dev_dataset):
            if len(batch[0]) == 0:
                break
            user_hist_rep = torch.tensor(news_rep[batch[1]], device = 'cuda') # (n_batch, n_hist)
            user = model.user_encoder(user_hist_rep).detach().cpu().numpy() # (n_batch, emb_dim)
            user_rep.append(user)
    # user_rep = np.concatenate(user_rep, axis = 0)
    return user_rep # [user_rep, ...]

In [16]:
args = {'model': 'NRMS', 'epochs': 6,
        'use_ent': True, 'ent_emb': 'avg'}
model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
news_rep = encode_all_news(model.news_encoder, news_info, news_ents) # (65238, 132) (65238, 5, 11)
user_rep = encode_all_user(model.user_encoder, dev_users, dev_user_hist, news_rep)

(65238, 132) (65238, 5, 11)
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]

torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torc

torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
to

torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torc

torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torc

...

torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
to

torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
to

torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torc

torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torc

torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
torch.Size([32, 256])
torch.Size([32, 132]) torch.Size([32, 5, 11])
to

## Model

In [12]:
class AttentionPooling(nn.Module):
    def __init__(self, emb_dim, query_dim):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim, query_dim)
        self.fc2 = nn.Linear(query_dim, 1)
        
    def forward(self, x, mask = None):
        '''
        (n_batch, n_seq, emb_dim) -> (n_batch, emb_dim)
        a = q^T tanh(V * k + v)
        alpha = softmax(a)
        '''
        a = self.fc2(torch.tanh(self.fc1(x))) # (n_batch, n_seq, 1)
        if mask is not None:
            a = a.masked_fill(mask.unsqueeze(-1) == 0, -1e9)
        alpha = F.softmax(a, dim = -2) # (n_batch, n_seq, 1)
        r = torch.matmul(alpha.transpose(-2, -1), x).squeeze(-2) # (n_batch, emb_dim)
        return r

In [13]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super().__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask=None):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k)
        scores = torch.exp(scores)
        if attn_mask is not None:
            scores = scores * attn_mask
        attn = scores / (torch.sum(scores, dim=-1, keepdim=True)  + 1e-8)
        
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_k, d_v):
        super().__init__()
        self.d_model = d_model # 300
        self.n_heads = n_heads # 20
        self.d_k = d_k # 20
        self.d_v = d_v # 20
        
        self.W_Q = nn.Linear(d_model, d_k * n_heads) # 300, 400
        self.W_K = nn.Linear(d_model, d_k * n_heads) # 300, 400
        self.W_V = nn.Linear(d_model, d_v * n_heads) # 300, 400
        
        self._initialize_weights()
                
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)
                
    def forward(self, Q, K, V, attn_mask=None):
        residual, batch_size = Q, Q.size(0)
        
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)
        
        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(1).expand(batch_size, max_len, max_len) 
            attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) 
        
        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s, attn_mask) 
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) 
        return context # (n_batch, n_seq, emb_dim)

In [14]:
# Self-Attn News Encoder (for NRMS)
class AttnNewsEncoder(nn.Module):
    def __init__(self, args, word_emb, cate_emb, ent_emb):
        super().__init__()
        self.args = args
        if 'ent_attn' not in self.args:
            self.args['ent_attn'] = False
        
        self.word_embedding = nn.Embedding.from_pretrained(word_emb)
        self.self_attn = MultiHeadSelfAttention(word_emb.shape[1], 16, 16, 16)
        self.addi_attn = AttentionPooling(256, 200)
        self.dropout = nn.Dropout(0.2)
        
        if self.args['use_ent']:
            if args['ent_emb'] == 'transe':
                self.ent_embedding = nn.Embedding.from_pretrained(ent_emb)
            elif args['ent_emb'] == 'random':
                self.ent_embedding = nn.Embedding(ent_emb.shape[0], ent_emb.shape[1])
            
            if args['ent_attn']:
                self.ent_transformer = MultiHeadSelfAttention(ent_emb.shape[1], 16, 16, 16)
            elif args['ent_emb'] == 'avg':
                self.ent_fc1 = nn.Linear(word_emb.shape[1], 200)
                self.ent_fc2 = nn.Linear(200, 256)
            else:
                self.ent_fc1 = nn.Linear(ent_emb.shape[1], 200)
                self.ent_fc2 = nn.Linear(200, 256)
            self.ent_attn = AttentionPooling(256, 200)
            self.aggr_fc = nn.Linear(512, 256)
        else:
            self.aggr_fc = nn.Linear(256, 256)
    
    def forward(self, news, ents = None):
        title, body, cate = news[:, :max_title], news[:, max_title: -2], news[:, -2:]
        ents, ent_words = ents[:, :, 0].squeeze(-1), ents[:, :, 1:]
        
        t_rep = self.word_embedding(title) # (n_batch, n_seq, emb_dim)
        t_rep = self.dropout(t_rep)
        t_rep = self.self_attn(t_rep, t_rep, t_rep) # (n_batch, n_seq, 256)
        t_rep = self.addi_attn(t_rep) # (n_batch, 256)
        
        if self.args['use_ent']:
            if self.args['ent_emb'] == 'avg':
                e_rep = self.word_embedding(ent_words) # (n_batch, n_ent, n_ent_word, emb_dim)
                e_rep = torch.mean(e_rep, dim = -2)
            else:
                e_rep = self.ent_embedding(ents) # (n_batch, n_ent, emb_dim)
                e_rep = self.dropout(e_rep)
            
            if self.args['ent_attn']:
                e_rep = self.ent_transformer(e_rep, e_rep, e_rep) # (n_batch, n_ent, 256)
            else:
                e_rep = F.relu(self.ent_fc1(e_rep))
                e_rep = self.ent_fc2(e_rep) # (n_news, n_ent, news_dim)
            e_rep = self.ent_attn(e_rep) # (n_batch, 256)
            t_rep = torch.cat((t_rep, e_rep), dim = -1)
        
        r = self.aggr_fc(t_rep)
        if 'aggr_relu' in self.args and self.args['aggr_relu']:
            r = F.relu(r)
        return r # (n_news, n_filter)

class AttnUserEncoder(nn.Module):
    def __init__(self, n_head, news_dim, query_dim):
        super().__init__()
        self.self_attn = MultiHeadSelfAttention(news_dim, n_head, 16, 16)
        self.addi_attn = AttentionPooling(news_dim, query_dim)
    
    def forward(self, h): # (n_batch, n_news, 256)
        u = self.self_attn(h, h, h) # (n_batch, n_news, 256)
        u = self.addi_attn(u) # (n_batch, 256)
        return u

In [15]:
class Model(nn.Module):
    def __init__(self, args, word_emb, cate_emb, ent_emb):
        super().__init__()
        if args['model'] == 'NAML':
            self.news_encoder = ConvNewsEncoder(args, word_emb, cate_emb, ent_emb)
            self.user_encoder = UserEncoder(256)
        if args['model'] == 'NRMS':
            n_head, query_dim, news_dim = 16, 200, 256
            self.news_encoder = AttnNewsEncoder(args, word_emb, cate_emb, ent_emb)
            self.user_encoder = AttnUserEncoder(n_head, news_dim, query_dim)
    
    def forward(self, hist, samp, samp_ents = None, user_ents = None):
        n_batch, n_news, n_sequence = hist.shape
        n_samp = samp.shape[1] # k + 1
        n_ents = samp_ents.shape[2]
        n_ent_words = samp_ents.shape[3] # n_words + 1
        
        hist = hist.reshape(n_batch * n_news, n_sequence)
        if user_ents is not None:
            user_ents = user_ents.reshape(n_batch * n_news, n_ents, n_ent_words)
        h = self.news_encoder(hist, user_ents) # (n_batch*n_news, n_filter)
        h = h.reshape(n_batch, n_news, -1)  # (n_batch, n_news, n_filter)
        u = self.user_encoder(h) # (n_batch, n_filter)
        
        samp = samp.reshape(n_batch * n_samp, n_sequence)
        if samp_ents is not None:
            samp_ents = samp_ents.reshape(n_batch * n_samp, n_ents, n_ent_words)
        r = self.news_encoder(samp, samp_ents) # (n_batch*(k+1), n_filter)
        r = r.reshape(n_batch, n_samp, -1) # (n_batch, k + 1, n_filter)
        
        y = torch.bmm(r, u.unsqueeze(2)) # (n_batch, K + 1, 1)
        return y.squeeze(2)

## train

In [16]:
def train_epoch(model, train_dataset, optimizer, entrophy):
    train_losses = []
    model.train()
    device = 'cuda'
    for _, batch in enumerate(train_dataset):
        if batch[0].shape[0] == 0:
            break
        sample = torch.tensor(batch[0], dtype = torch.long, device = device)
        history = torch.tensor(batch[1], dtype = torch.long, device = device)
        correct = torch.argmax(torch.tensor(batch[2], dtype = torch.long, device = device), dim = 1)
        samp_ents = torch.tensor(batch[3], dtype = torch.long, device = device)
        user_ents = torch.tensor(batch[4], dtype = torch.long, device = device)
        optimizer.zero_grad()
        output = model(history, sample, samp_ents, user_ents)
        loss = entrophy(output, correct)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.average(train_losses)

In [17]:
def evaluate(model, dev_dataset, news_info, dev_users, dev_user_hist, news_ents = None):
    print(news_info.shape, news_ents.shape)
    news_rep = encode_all_news(model.news_encoder, news_info, news_ents) # (65238, 132) (65238, 5, 11)
    user_rep = encode_all_user(model.user_encoder, dev_users, dev_user_hist, news_rep)
    
    model.eval()
    with torch.no_grad():
        auc_scores = []
        mrr_scores = []
        ndcg5_scores = []
        ndcg10_scores = []
        for i, batch in enumerate(dev_dataset):
            if len(batch[0]) == 0:
                break
            user = user_rep[i]
            for j in range(len(batch[0])):
                sample = news_rep[batch[0][j]] # (n_imp, emb_dim)
                positive = batch[2][j] # (1, n_imp)

                score = np.matmul(sample, user[j]) # (1, n_imp)
                predict = np.exp(score) / np.sum(np.exp(score))

                auc_scores.append(roc_auc_score(positive, predict))
                mrr_scores.append(mrr_score(positive, predict))
                ndcg5_scores.append(ndcg_score(positive, predict, k = 5))
                ndcg10_scores.append(ndcg_score(positive, predict, k = 10))
    return np.mean(auc_scores), np.mean(mrr_scores), np.mean(ndcg5_scores), np.mean(ndcg10_scores)

In [18]:
def train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents = None, epochs = 4):
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    entrophy = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        begin_time = time.time()
        loss = train_epoch(model, train_dataset, optimizer, entrophy)
        auc, mrr, ndcg5, ndcg10 = evaluate(model, dev_dataset, news_info, dev_users, dev_user_hist, news_ents)
        end_time = time.time()
        print('[epoch {:d}] loss: {:.4f}, AUC: {:.4f}, MRR: {:.4f}, nDCG5:{:.4f}, nDCG10: {:.4f}, Time: {:.2f}'.format(
            epoch + 1, loss, auc, mrr, ndcg5, ndcg10, end_time - begin_time))
    return auc, mrr, ndcg5, ndcg10

In [19]:
def train(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents = None, epochs = 6):
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    entrophy = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        begin_time = time.time()
        loss = train_epoch(model, train_dataset, optimizer, entrophy)
        end_time = time.time()
    auc, mrr, ndcg5, ndcg10 = evaluate(model, dev_dataset, news_info, dev_users, dev_user_hist, news_ents)
    return auc, mrr, ndcg5, ndcg10

In [20]:
def train_multi_times(args, word_emb, cate_emb, ent_emb, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents = None):
    print(args)
    aucs, mrrs, ndcg5s, ndcg10s = [], [], [], []
    for i in range(3):
        model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
        auc, mrr, ndcg5, ndcg10 = train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents, 6)
        aucs.append(auc)
        mrrs.append(mrr)
        ndcg5s.append(ndcg5)
        ndcg10s.append(ndcg10)
    auc, mrr, ndcg5, ndcg10 = np.average(aucs), np.average(mrrs), np.average(ndcg5s), np.average(ndcg10s)
    print(auc, mrr, ndcg5, ndcg10)
    print('Average AUC: {:.4f} , MRR: {:.4f}, nDCG5:{:.4f}, nDCG10: {:.4f}'.format(auc, mrr, ndcg5, ndcg10))
    return auc, mrr, ndcg5, ndcg10

In [66]:
model = Model(args, word_emb, cate_emb, ent_emb)
print(model)

Model(
  (news_encoder): AttnNewsEncoder(
    (word_embedding): Embedding(82568, 300)
    (self_attn): MultiHeadSelfAttention(
      (W_Q): Linear(in_features=300, out_features=256, bias=True)
      (W_K): Linear(in_features=300, out_features=256, bias=True)
      (W_V): Linear(in_features=300, out_features=256, bias=True)
    )
    (addi_attn): AttentionPooling(
      (fc1): Linear(in_features=256, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1, bias=True)
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (ent_fc1): Linear(in_features=300, out_features=200, bias=True)
    (ent_fc2): Linear(in_features=200, out_features=256, bias=True)
    (ent_attn): AttentionPooling(
      (fc1): Linear(in_features=256, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1, bias=True)
    )
    (aggr_fc): Linear(in_features=512, out_features=256, bias=True)
  )
  (user_encoder): AttnUserEncoder(
    (self_attn): MultiHeadSelfAttention(

In [21]:
args = {'model': 'NRMS', 'epochs': 6,
        'use_ent': True, 'ent_emb': 'avg'}
model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents, 10)
model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents, 10)

[epoch 1] loss: 1.4424, AUC: 0.6418, MRR: 0.2945, nDCG5:0.3244, nDCG10: 0.3889, Time: 525.86
[epoch 2] loss: 1.3600, AUC: 0.6606, MRR: 0.3065, nDCG5:0.3389, nDCG10: 0.4023, Time: 531.42
[epoch 3] loss: 1.3323, AUC: 0.6599, MRR: 0.3046, nDCG5:0.3347, nDCG10: 0.4003, Time: 527.28
[epoch 4] loss: 1.3132, AUC: 0.6621, MRR: 0.3104, nDCG5:0.3421, nDCG10: 0.4061, Time: 526.55
[epoch 5] loss: 1.2992, AUC: 0.6645, MRR: 0.3118, nDCG5:0.3420, nDCG10: 0.4067, Time: 530.80
[epoch 6] loss: 1.2870, AUC: 0.6678, MRR: 0.3136, nDCG5:0.3468, nDCG10: 0.4096, Time: 527.72
[epoch 7] loss: 1.2756, AUC: 0.6588, MRR: 0.3096, nDCG5:0.3399, nDCG10: 0.4048, Time: 528.58
[epoch 8] loss: 1.2650, AUC: 0.6656, MRR: 0.3145, nDCG5:0.3477, nDCG10: 0.4109, Time: 526.91
[epoch 9] loss: 1.2544, AUC: 0.6655, MRR: 0.3125, nDCG5:0.3441, nDCG10: 0.4083, Time: 526.82
[epoch 10] loss: 1.2448, AUC: 0.6652, MRR: 0.3128, nDCG5:0.3466, nDCG10: 0.4092, Time: 525.60
[epoch 1] loss: 1.4423, AUC: 0.6428, MRR: 0.2941, nDCG5:0.3254, nDCG1

KeyboardInterrupt: 

In [21]:
args = {'model': 'NRMS', 'epochs': 6, 'use_ent': True, 'ent_emb': 'transe'}
aucs, mrrs, ndcg5s, ndcg10s = [], [], [], []
for i in range(3):
    model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
    auc, mrr, ndcg5, ndcg10 = train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents, 6)
    aucs.append(auc)
    mrrs.append(mrr)
    ndcg5s.append(ndcg5)
    ndcg10s.append(ndcg10)
auc, mrr, ndcg5, ndcg10 = np.average(aucs), np.average(mrrs), np.average(ndcg5s), np.average(ndcg10s)
print(auc, mrr, ndcg5, ndcg10)
print('Average AUC: {:.4f} , MRR: {:.4f}, nDCG5:{:.4f}, nDCG10: {:.4f}'.format(auc, mrr, ndcg5, ndcg10))


(65238, 132) (65238, 5, 11)


/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [124,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [125,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [126,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [127,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [118,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/

RuntimeError: CUDA error: device-side assert triggered

In [25]:
args = {'model': 'NRMS', 'epochs': 6, 'use_ent': True, 'ent_emb': 'transe'}
auc, mrr, ndcg5, ndcg10 = train_multi_times(args, word_emb, cate_emb, ent_emb, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents)

{'model': 'NRMS', 'epochs': 6, 'use_ent': True, 'ent_emb': 'transe'}


/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [124,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [125,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [126,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [117,0,0], thread: [127,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/cuda/Indexing.cu:658: indexSelectLargeIndex: block: [118,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/

RuntimeError: CUDA error: device-side assert triggered

In [26]:
model


Model(
  (news_encoder): AttnNewsEncoder(
    (word_embedding): Embedding(82568, 300)
    (self_attn): MultiHeadSelfAttention(
      (W_Q): Linear(in_features=300, out_features=256, bias=True)
      (W_K): Linear(in_features=300, out_features=256, bias=True)
      (W_V): Linear(in_features=300, out_features=256, bias=True)
    )
    (addi_attn): AttentionPooling(
      (fc1): Linear(in_features=256, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1, bias=True)
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (ent_embedding): Embedding(31453, 100)
    (ent_fc1): Linear(in_features=100, out_features=200, bias=True)
    (ent_fc2): Linear(in_features=200, out_features=256, bias=True)
    (ent_attn): AttentionPooling(
      (fc1): Linear(in_features=256, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1, bias=True)
    )
    (aggr_fc): Linear(in_features=512, out_features=256, bias=True)
  )
  (user_encoder): AttnUserEncode

In [55]:
args = {'model': 'NRMS', 'epochs': 6,
        'use_ent': True, 'ent_emb': 'avg'}
model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents, 6)

[epoch 1] loss: 1.4388, AUC: 0.6439, MRR: 0.2971, nDCG5:0.3272, nDCG10: 0.3918, Time: 524.60
[epoch 3] loss: 1.3297, AUC: 0.6530, MRR: 0.3015, nDCG5:0.3332, nDCG10: 0.3977, Time: 524.80
[epoch 4] loss: 1.3111, AUC: 0.6592, MRR: 0.3072, nDCG5:0.3404, nDCG10: 0.4039, Time: 526.89
[epoch 5] loss: 1.2965, AUC: 0.6546, MRR: 0.3023, nDCG5:0.3352, nDCG10: 0.3993, Time: 527.75
[epoch 6] loss: 1.2835, AUC: 0.6642, MRR: 0.3098, nDCG5:0.3430, nDCG10: 0.4067, Time: 529.43


In [ ]:
args = {'model': 'NRMS', 'epochs': 6,
        'use_ent': True, 'ent_emb': 'random'}
model = Model(args, word_emb, cate_emb, ent_emb).to('cuda')
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, news_ents, 6)